迭代器和生成器
- 可迭代对象和迭代器

容器：<br>
Python中一切皆对象，对象的抽象是类，对象的集合是容器。
- 列表、元组、字典、集合都是容器
- 容器都是可以迭代的
- 迭代器（iterator）提供一个next方法。
- 可迭代对象可以通过iter()函数返回一个迭代器，再通过next()函数就可以实现遍历。
- for in 语句将这个过程隐式化。

In [2]:
from collections.abc import Iterable


# 判断一个对象是否可迭代
def is_iterable(param):
    try:
        iter(param)
        return True
    except TypeError:
        return False
# 或者


def isNot_iterable(param):
    return isinstance(param, Iterable)


parms = [1234, '1234', [1, 2, 3, 4], set(
    [1, 2, 3, 4]), {1: 1, 2: 2, 3: 3, 4: 4}, (1, 2, 3, 4)]

for param in parms:
    print('{} is iterable? {} {}'.format(
        param, is_iterable(param), isNot_iterable(param)))


1234 is iterable? False False
1234 is iterable? True True
[1, 2, 3, 4] is iterable? True True
{1, 2, 3, 4} is iterable? True True
{1: 1, 2: 2, 3: 3, 4: 4} is iterable? True True
(1, 2, 3, 4) is iterable? True True


-<br>
- 生成器<br>
懒人版本的迭代器


In [12]:
# 在迭代器中，如果我们想要枚举它的元素，这些元素要事先生成。举例如下：
import os
import psutil

# 显示当前python程序占用的内存大小


def show_memory_info(hint):
    pid = os.getpid()
    p = psutil.Process(pid)

    info = p.memory_full_info()
    memory = info.uss/1024./1024
    print('{} memory used: {} MB'.format(hint, memory))


def test_iterator():
    show_memory_info('initing iterator')
    list_1 = [i for i in range(100000000)]
    # list_1=iter(list_1)
    show_memory_info('after iterator initiated')
    print(sum(list_1))
    show_memory_info('after sum called')


def test_generator():
    show_memory_info('initing generator')
    list_2 = (i for i in range(100000000))
    show_memory_info('after generator initiated')
    print(sum(list_2))
    show_memory_info('after sum called')


%time test_iterator()
%time test_generator()


initing iterator memory used: 20.9765625 MB
after iterator initiated memory used: 3891.31640625 MB
4999999950000000
after sum called memory used: 3891.81640625 MB
CPU times: total: 13.4 s
Wall time: 14.2 s
initing generator memory used: 20.91796875 MB
after generator initiated memory used: 20.91796875 MB
4999999950000000
after sum called memory used: 20.921875 MB
CPU times: total: 11.6 s
Wall time: 11.9 s


迭代器，生成的元素都会保存到内存中，占用巨量的内存，内存不够可能会出现OOM错误。<br>
有时候，我们并不需要在内存中保存这么多东西，比如对元素和，我们只知道每个元素在相加的那一刻是多少就行了，用完就可以扔掉了。<br>
于是，生成器的概念应运而生，在调用next()函数的时候，才会生成下一个变量。


生成器，还能玩出什么花样？
- 数学中有一个恒等式，(1+2+3+...+n)^2=1^3+2^3+3^3+...+n^3,用代码来验证一下。

In [4]:
def generator(k):
    i = 1
    while True:
        yield i ** k
        i = i + 1


gen_1 = generator(1)
gen_3 = generator(3)
print(gen_1)
print(gen_3)


def get_sum(n):
    sum_1, sum_3 = 0, 0
    for _ in range(n):
        next_1 = next(gen_1)
        next_3 = next(gen_3)
        print('next_1 = {}, next_3 = {}'.format(next_1, next_3))
        sum_1 += next_1
        sum_3 += next_3
    print(sum_1*sum_1, sum_3)


get_sum(8)


<generator object generator at 0x0000019F3F782DD0>
<generator object generator at 0x0000019F3F782F90>
next_1 = 1, next_3 = 1
next_1 = 2, next_3 = 8
next_1 = 3, next_3 = 27
next_1 = 4, next_3 = 64
next_1 = 5, next_3 = 125
next_1 = 6, next_3 = 216
next_1 = 7, next_3 = 343
next_1 = 8, next_3 = 512
1296 1296


给定一个list和一个指定数字，求这个数字在list的位置。

In [5]:
# 常规做法
def index_normal(L, target):
    result = []
    for i, num in enumerate(L):
        if num == target:
            result.append(i)
    return result


print(index_normal([1, 6, 2, 4, 5, 2, 8, 6, 3, 2], 2))


[2, 5, 9]


In [7]:
# 迭代器做法
def index_generator(L, target):
    for i, num in enumerate(L):
        if num == target:
            yield i


gen_1 = index_generator([1, 6, 2, 4, 5, 2, 8, 6, 3, 2], 2)

print(list(gen_1))


[2, 5, 9]


给定两个序列，判断第一个是不是第二个的子序列。

In [8]:
def is_subsequence(a, b):
    b = iter(b)
    return all(i in b for i in a)


print(is_subsequence([1, 3, 5], [1, 2, 3, 4, 5]))
print(is_subsequence([1, 4, 3], [1, 2, 3, 4, 5]))

# (i in b )等效于
# while True:
#     val=next(b)
#     if val==i:
#         yield True

True
False


In [15]:
def is_subsequence(a, b):
    b = iter(b)
    print(b)

    gen = (i for i in a)
    print(gen)

    for i in gen:
        print(i)

    gen = ((i in b) for i in a)
    print(gen)

    for i in gen:
        print(i)

    # 迭代器b已经迭代结束
    return all(((i in b) for i in a))


print('one',is_subsequence([1, 3, 5], [1, 2, 3, 4, 5]))
print('two',is_subsequence([1, 4, 3], [1, 2, 3, 4, 5]))


<generator object is_subsequence.<locals>.<genexpr> at 0x0000019F3F7BACF0>
1
3
5
<generator object is_subsequence.<locals>.<genexpr> at 0x0000019F3F7BA660>
True
True
True
one False
<generator object is_subsequence.<locals>.<genexpr> at 0x0000019F3F7BA660>
1
4
3
<generator object is_subsequence.<locals>.<genexpr> at 0x0000019F3F7BACF0>
True
True
False
two False



![这是图片](../../image/00/generator.png "generator")